In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("/kaggle/input/cse-281-24-predict-the-item-price/train.csv")
df_test = pd.read_csv("/kaggle/input/cse-281-24-predict-the-item-price/test.csv")

# Preprocessing steps
df['X8'] = 2024 - df['X8']

df['X3'].replace({'LF': 'Low Fat', 'low fat': 'Low Fat', 'reg': 'Regular'}, inplace=True)
df_test['X3'].replace({'LF': 'Low Fat', 'low fat': 'Low Fat', 'reg': 'Regular'}, inplace=True)

imputer = KNNImputer(n_neighbors=5)
df['X2'] = imputer.fit_transform(df[['X2']])
df_test['X2'] = imputer.transform(df_test[['X2']])

mode_X9 = df['X9'].mode()[0]
df['X9'].fillna(mode_X9, inplace=True)
df_test['X9'].fillna(mode_X9, inplace=True)

df['X1'] = df['X1'].apply(lambda x: x[:2])
df_test['X1'] = df_test['X1'].apply(lambda x: x[:2])
df.replace({'X1': {'FD': 'Food', 'DR': 'Drink', 'NC': 'Non Consumable'}}, inplace=True)
df_test.replace({'X1': {'FD': 'Food', 'DR': 'Drink', 'NC': 'Non Consumable'}}, inplace=True)

#Label encode the categorical columns that have an order
label_encoder = LabelEncoder()
df['X9'] = label_encoder.fit_transform(df['X9'])
df_test['X9'] = label_encoder.transform(df_test['X9'])
df['X10'] = label_encoder.fit_transform(df['X10'])
df_test['X10'] = label_encoder.transform(df_test['X10'])
df['X11'] = label_encoder.fit_transform(df['X11'])
df_test['X11'] = label_encoder.transform(df_test['X11'])

columns_to_encode = ['X1', 'X3', 'X5', 'X7']
df = pd.get_dummies(df, columns=columns_to_encode, prefix_sep='_')
df_test = pd.get_dummies(df_test, columns=columns_to_encode, prefix_sep ='_')

#We Scale the numerical columns, as linear reg requires scaling for optimum predictions
features_to_scale = ['X2', 'X4', 'X6', 'X8' ,'X9' ,'X10' ,'X11' ]
scaler = MinMaxScaler()
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])
df_test[features_to_scale] = scaler.transform(df_test[features_to_scale])

features = []
for col in df.columns:
    if df[col].corr(df['Y']) >= 0.25 or df[col].corr(df['Y']) <= -0.25:
        features.append(col)
features.remove('Y')

X = df[features]
y = df['Y']
x_test = df_test[features]

linear_model = LinearRegression()
linear_model.fit(X, y)
y_pred = linear_model.predict(x_test)

submission = pd.DataFrame({'Y': y_pred})
submission['row_id'] = range(submission.shape[0])
submission = submission[['row_id', 'Y']]
submission.to_csv('submission.csv', index=False)

from sklearn.model_selection import train_test_split

X = df[features]
y = df['Y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

y_pred_test = linear_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_test)
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 0.4425440830298246
